In [1]:
!pip install pandas openpyxl

In [9]:
#%% Importar bibliotecas necessárias
import pandas as pd
import os

#%% Definir parâmetros iniciais
input_dir = r'input'  # Atualize este caminho
output_file = 'consolidado_filtrado.xlsx'
coluna_filtro = 'STATUS DO PROFISSIONAL'  # Removido os colchetes para ser uma string
valores_filtro = ['Licença Maternidade', 'Licença Médica - acima de 15 dias']

#%% Processamento em lote
consolidado = pd.DataFrame()

for arquivo in os.listdir(input_dir):
    if arquivo.endswith(('.xlsx', '.xls')):
        print(f'Processando: {arquivo}...')
        
        # Carregar arquivo
        caminho_completo = os.path.join(input_dir, arquivo)
        df = pd.read_excel(caminho_completo)
        
        # Verificar se a coluna existe
        if coluna_filtro not in df.columns:  # Agora verificando uma string, não uma lista
            print(f'Aviso: Arquivo {arquivo} não possui a coluna {coluna_filtro}')
            continue
            
        # Aplicar filtro
        df_filtrado = df[df[coluna_filtro].isin(valores_filtro)]
        
        # Adicionar ao consolidado
        consolidado = pd.concat([consolidado, df_filtrado], ignore_index=True)

#%% Exportar resultados
if not consolidado.empty:
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        consolidado.to_excel(writer, index=False)
    print(f'\nArquivo consolidado salvo em: {output_file}')
    print(f'Total de registros filtrados: {len(consolidado):,}')
else:
    print('Nenhum registro encontrado com os critérios especificados')

Processando: 01_Janeiro_2025.xlsx...
Processando: 02_Fevereiro_2025.xlsx...
Processando: 03_Marco_2025.xlsx...

Arquivo consolidado salvo em: consolidado_filtrado.xlsx
Total de registros filtrados: 1,325


In [13]:
import pandas as pd

#Antes de executar, o arquivo .xlsx  deve estar no diretório raiz do Jupyterlab
#A primeira linha da planilha não pode estar mesclada, copiar e colar conteúdo em outra planilha como Texto


# Função para extrair a última data e formatá-la
def extrair_ultima_data(data_str):
    if isinstance(data_str, str):
        datas = [d.strip() for d in data_str.split('|') if d.strip()]
        if datas:
            ultima_data = datas[-1]
            try:
                # Tenta converter para datetime e retorna no formato desejado
                return pd.to_datetime(ultima_data, format='%d/%m/%Y').strftime('%d/%m/%Y')
            except:
                return None  # Caso a data seja inválida
    return data_str

# Carregar os dados
df = pd.read_excel("consolidado_filtrado.xlsx", skiprows=0, decimal=',')

# Remover coluna CID
df = df.drop(columns=['CID'])

# Processar colunas de data
for coluna in ['DATA INICIO', 'DATA FIM']:
    df[coluna] = df[coluna].apply(extrair_ultima_data)

# Salvar o arquivo tratado
df.to_excel("consolidado_filtrado_tratado.xlsx", index=False)

print("Processamento concluído! Formato das datas:")
print(df[['DATA INICIO', 'DATA FIM']].head())

Processamento concluído! Formato das datas:
  DATA INICIO    DATA FIM
0  29/10/2024  25/02/2025
1  22/10/2024  18/02/2025
2  08/01/2025  09/03/2025
3  26/11/2024  26/01/2025
4  30/10/2024  30/04/2025
